<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Risk Parity Over Time

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

### The use of the "Python 3.10, Numpy 1.26.4" kernel is recommended.

## Out-of-Sample Testing

Topics of interest include:

* out-of-sample performance
* risk parity portfolio
 * without target volatility
 * with target volatility

## Real Financial Data

**_Historical end-of-day financial time series data._**

See AI in Finance (ch. 04) and `http://hilpisch.com/aiif_eikon_eod_data.csv`.

## Imports and Data

In [ ]:
import cufflinks
import numpy as np
import pandas as pd
from pylab import plt, mpl
from risk_measures import *
from scipy.optimize import minimize
cufflinks.set_config_file(offline=True)
plt.style.use('seaborn-v0_8')
np.set_printoptions(precision=5, suppress=True,
                   formatter={'float': lambda x: f'{x:6.3f}'})

In [ ]:
url = 'http://hilpisch.com/aiif_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
# raw.info()

In [ ]:
symbols = raw.columns[:5]
# symbols = ['SPY', 'GLD', 'EUR=']
symbols

In [ ]:
data = raw[symbols]

In [ ]:
noa = len(symbols)

In [ ]:
data.normalize().iplot()

In [ ]:
rets = np.log(data / data.shift(1)).dropna()

## Performance Out-of-Sample

In [ ]:
bnds = noa * [(0, 1),]

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
w = np.array(noa * [1 / noa])

In [ ]:
target = np.array(noa * [1 / noa])

In [ ]:
def rel_risk_contributions(weights, rets=rets):
    vol = portfolio_volatility(weights, rets)
    cov = rets.cov()
    mvols = np.dot(cov, weights) / vol
    rc = mvols * weights
    rc = rc / rc.sum()
    return rc

In [ ]:
def mse_risk_contributions(weights, target, rets=rets):
    rc = rel_risk_contributions(weights, rets)
    mse = ((rc - target) ** 2).mean()
    return mse * 100

In [ ]:
rel_risk_contributions(w)

In [ ]:
mse_risk_contributions(w, target)

In [ ]:
opt_weights = {}
for year in range(2010, 2019):
    rets_ = rets.loc[f'{year - 1}-01-01':f'{year}-12-31']
    ow = minimize(lambda w: mse_risk_contributions(
                    w, target=target, rets=rets_),
                    w, bounds=bnds, constraints=cons)['x']
    opt_weights[year] = ow

In [ ]:
opt_weights

In [ ]:
res = pd.DataFrame()
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year}-01-01':f'{year}-12-31']
    epv = portfolio_volatility(opt_weights[year], rets_)
    epr = portfolio_return(opt_weights[year], rets_)
    esr = epr / epv
    rets_ = rets[symbols].loc[f'{year + 1}-01-01':f'{year + 1}-12-31']
    rpv = portfolio_volatility(opt_weights[year], rets_)
    rpr = portfolio_return(opt_weights[year], rets_)
    rsr = rpr / rpv
    res = pd.concat([res, pd.DataFrame({'epv': epv, 'epr': epr, 'esr': esr,
                                   'rpv': rpv, 'rpr': rpr, 'rsr': rsr},
                                  index=[year + 1])])

In [ ]:
res

In [ ]:
res.mean()

In [ ]:
(res[['rpv', 'rpr', 'rsr']].mean().values -
 res[['epv', 'epr', 'esr']].mean().values)

In [ ]:
res[['epv', 'rpv']].corr()

In [ ]:
res[['epv', 'rpv']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Volatility');

In [ ]:
res[['epr', 'rpr']].corr()

In [ ]:
res[['epr', 'rpr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Return');

In [ ]:
res[['esr', 'rsr']].corr()

In [ ]:
res[['esr', 'rsr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Sharpe Ratio');

## Target Volatility

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights:
        portfolio_volatility(weights, rets_) - 0.15}

In [ ]:
opt_weights = {}
for year in range(2010, 2019):
    rets_ = rets.loc[f'{year - 1}-01-01':f'{year}-12-31']
    opt = minimize(lambda weights: mse_risk_contributions(
                    weights, target=target, rets=rets_),
                    w, bounds=None, constraints=cons)
    # print(portfolio_volatility(opt['x'], rets_))
    opt_weights[year] = opt['x']

In [ ]:
opt_weights

In [ ]:
opt_weights[2012].sum()

In [ ]:
res = pd.DataFrame()
for year in range(2010, 2019):
    rets_ = rets[symbols].loc[f'{year}-01-01':f'{year}-12-31']
    epv = portfolio_volatility(opt_weights[year], rets_)
    epr = portfolio_return(opt_weights[year], rets_)
    esr = epr / epv
    rets_ = rets[symbols].loc[f'{year + 1}-01-01':f'{year + 1}-12-31']
    rpv = portfolio_volatility(opt_weights[year], rets_)
    rpr = portfolio_return(opt_weights[year], rets_)
    rsr = rpr / rpv
    res = pd.concat([res, pd.DataFrame({'epv': epv, 'epr': epr, 'esr': esr,
                                   'rpv': rpv, 'rpr': rpr, 'rsr': rsr},
                                  index=[year + 1])])

In [ ]:
res

In [ ]:
res.mean()

In [ ]:
(res[['rpv', 'rpr', 'rsr']].mean().values -
 res[['epv', 'epr', 'esr']].mean().values)

In [ ]:
res[['epv', 'rpv']].corr()

In [ ]:
res[['epv', 'rpv']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Volatility');

In [ ]:
res[['epr', 'rpr']].corr()

In [ ]:
res[['epr', 'rpr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Portfolio Return');

In [ ]:
res[['esr', 'rsr']].corr()

In [ ]:
res[['esr', 'rsr']].plot(kind='bar', figsize=(10, 6),
        title='Expected vs. Realized Sharpe Ratio');

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>